<a href="https://colab.research.google.com/github/Ayush123-prog/Chatbot/blob/master/acmegrade_ass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
lemmatizer = WordNetLemmatizer()
words = []
classes=[]
doc_x=[]
doc_y=[]

In [6]:
for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenizer(pattern)
    words.extend(tokens)
    doc_x.append(pattern)
    doc_y.append(intent["tag"])
if intent["tag"] not in classes:
  classes.append(intent["tag"])

NameError: name 'data' is not defined

In [7]:
words = [lemmatizer.lemmatize(word.lower() for word in words if word not in string.punctuation)]
words = sorted(set(words))
classes = sorted(set(classes))
print(words)
print(classes)
print(doc_x)
print(doc_y)

AttributeError: 'generator' object has no attribute 'endswith'

In [ ]:
training = []
out_empty = [0]*len(classes)
for idx,doc in enumerate(doc_x):
  bow=[]
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.apppend(1) if word in text else bow.append (0)
  output_row = list(out_empty)
  output_row[classes.index(doc_y[idx])] = 1
  training.append([bow,output_row])
random.shuffle(training)
training = np.array(training, dtype= object)

train_X = np.array(list(training[:,0]))
train_Y = np.array(list(training[:,1]))

In [ ]:
input_shape = (len(train_X[0]),)
output_shape = len(train_Y[0])

epochs = 500

In [ ]:
from tensorfow.keras.models import Seqential
from tensorflow.keras.layers import Dense,Dropout
model = Sequential()
model.add(Dense(128,input_shape=input_shape,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(output_shape,activation='relu'))

adam = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])
print(model.summary())

In [ ]:
model.fit(x=train_X, y=train_Y, epochs =500, verbose=1)

In [ ]:
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens
def bag_of_words(text,vocab):
  tokens= clean_text(text)
  bow = [0]*len(vocab)
  for w in tpkens:
    for idx,word in enumerate(vocab):
      if word==w:
        bow[index]=1
  return np.array(bow)


In [ ]:
def pred_class(text,vocab,labels):
  bow = bag_of_words(text,vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx,res] for idx,res in enumerate(result) if res>thresh]

  y_pred.sort(key = lambda x:x[1], reverse=True)
  return_list=[]
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list
def get_response(intents_list, intents_json):
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents:
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
while True:
  message = input(" ")
  intents = pred_class(message, words, classes)
  result = get_response(intents, data)
  print(result)